### Basic library imports

In [1]:
import os
import pandas as pd

### Read Dataset

In [2]:
DATASET_FOLDER = '../dataset/'
train = pd.read_csv(os.path.join(DATASET_FOLDER, 'train.csv'))
test = pd.read_csv(os.path.join(DATASET_FOLDER, 'test.csv'))
sample_test = pd.read_csv(os.path.join(DATASET_FOLDER, 'sample_test.csv'))
sample_test_out = pd.read_csv(os.path.join(DATASET_FOLDER, 'sample_test_out.csv'))

### Run Sanity check using src/sanity.py

In [ ]:
!python sanity.py --test_filename ../dataset/sample_test.csv --output_filename ../dataset/sample_test_out.csv

In [ ]:
!python sanity.py --test_filename ../dataset/sample_test.csv --output_filename ../dataset/sample_test_out_fail.csv

### Download images

In [ ]:
from utils import download_images
download_images(sample_test['image_link'], '../images')

In [ ]:
# Justin Pytesseract
from utils import process_images_with_ocr_justin

del_ocr_results = process_images_with_ocr_justin('../del_images2')
print(del_ocr_results)

for image_name, text in del_ocr_results.items():
    print(f"Text extracted from {image_name}: {text}")


In [ ]:
# easyocr
from utils import process_images_with_easyocr

del_ocr_results = process_images_with_easyocr('../images') #directly pass the path of images directory as argument
# print(del_ocr_results)

for image_name, text in del_ocr_results.items():
    print(f"Text extracted from {image_name}: {text}")


In [ ]:
# Shivansh pytesseract
from utils import process_images_with_ocr

ocr_results = process_images_with_ocr(sample_test['image_link'], '../images')

for image_name, text in ocr_results.items():
    print(f"Text extracted from {image_name}: {text}")

In [ ]:
assert len(os.listdir('../images')) > 0

In [15]:
rm -rf ../images